In [ ]:
force_change = True  # Set to false to wait
min_auto_seconds = 58  # Seconds to wait

In [ ]:
import io
import json
import logging
import os
import random
import subprocess
import sys
import time
from datetime import datetime
from functools import lru_cache
from math import gcd
from pathlib import Path

import astral
import astral.sun
import psutil
import pytz
import requests
from bs4 import BeautifulSoup
from PIL import Image, ImageDraw, ImageEnhance


# Function to reduce a fraction
# to its lowest form
@lru_cache
def reduceFraction(x, y):
    d = gcd(x, y)

    x = x // d
    y = y // d
    return x, y

In [ ]:
wallpaper_dir = os.path.expanduser("~/Pictures/Wallpapers/")

In [ ]:
if not force_change:
    if (time.time() - os.stat("randomWp.log").st_mtime) < min_auto_seconds:
        sys.exit(0)

In [ ]:
##Magic Logging Bootstrap
logging.basicConfig(
    filename="randomWp.log",
    level=logging.INFO,
    format="%(asctime)s.%(msecs)03d %(levelname)s %(module)s - %(funcName)s: %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.getLogger().addHandler(logging.StreamHandler(sys.stdout))

exlogger = logging.getLogger("Exception")


def handle_exception(exc_type, exc_value, exc_traceback):
    if issubclass(exc_type, KeyboardInterrupt):
        sys.__excepthook__(exc_type, exc_value, exc_traceback)
        return
    exlogger.error("Uncaught exception", exc_info=(exc_type, exc_value, exc_traceback))


sys.excepthook = handle_exception

In [ ]:
def downloadSomePapers(size=None):
    resolution = ""
    if size is not None:
        resolution = f"&atleast={size}"
    if os.path.exists("wp-downloading"):
        if (time.time() - os.stat("wp-downloading").st_mtime) > 3 * 60:
            os.unlink("wp-downloading")
        else:
            print(
                f'wp-downloading file is present and good for {(3*60)-(time.time() - os.stat("wp-downloading").st_mtime)} seconds more'
            )
            sys.exit(1)

    with open("wp-downloading", "w") as f:
        f.write(" ")

    sfw = 1
    sketchy = 0
    nsfw = 0
    blacklist = ["https://wallhaven.cc/images/layout/logo_sm.png"]
    wallpaper_dir = os.path.expanduser("~/Pictures/Wallpapers")
    if not os.path.exists(wallpaper_dir):
        os.mkdir(wallpaper_dir)

    def process4WalledPage(url):
        res = requests.get(
            url,
            headers={
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"
            },
        )
        soup = BeautifulSoup(res.text, features="lxml")
        for item in soup.find_all("img"):
            if "data-src" in item.attrs:
                if item.attrs["data-src"] not in blacklist:
                    item = item.attrs["data-src"]
                    item = item.replace("https://th.wallhaven.cc/small/", "")
                    item = item.replace("/", "/wallhaven-")
                    url = f"https://w.wallhaven.cc/full/{item}"
                    fn = item.split("/").pop()
                    yield ((fn, url))

    random_url = f"https://wallhaven.cc/search?categories=111&purity={sfw}{sketchy}{nsfw}&atleast={resolution}&sorting=random&order=desc&page=3"
    print(f"scraping {random_url}")
    for filename, url in process4WalledPage(random_url):
        print(f"{url} => {filename}")
        res = requests.get(url)
        if res.status_code == 200 and len(res.content) > 1:
            img = Image.open(io.BytesIO(res.content))
            height, width = img.size
            size_str = f"{height}x{width}"
            print(f"adding image to: {size_str}")
            size_folder = Path(os.path.join(wallpaper_dir, size_str))
            if not size_folder.exists():
                size_folder.mkdir()
            out_path = os.path.join(wallpaper_dir, size_str, filename)
            with open(out_path, "wb") as f:
                f.write(res.content)
    os.unlink("wp-downloading")

In [ ]:
def get_DBUS_SESSION_BUS_ADDRESS():
    PID = None
    for p in psutil.process_iter():
        if p.name() == "gnome-session-binary":
            PID = p.pid
            break

    logging.debug(PID)

    DBUS_SESSION_BUS_ADDRESS = None
    with open(f"/proc/{PID}/environ", "rb") as f:
        d = f.read()
        for line in d.split(b"\x00"):
            if b"DBUS_SESSION_BUS_ADDRESS" in line:
                line = line.split(b"=", maxsplit=1)
                DBUS_SESSION_BUS_ADDRESS = line[1].decode("utf-8")
    logging.debug(DBUS_SESSION_BUS_ADDRESS)
    os.environ["DBUS_SESSION_BUS_ADDRESS"] = DBUS_SESSION_BUS_ADDRESS
    return DBUS_SESSION_BUS_ADDRESS

In [ ]:
def getSunset():
    latitude = 42.703750
    longitude = -84.438410
    tz_chicago = pytz.timezone("America/Chicago")
    tz_name = "America/Chicago"
    for_date = datetime.now().date()
    locationInfoObj = astral.LocationInfo(
        "Custom Name", "My Region", tz_name, latitude, longitude
    )
    s = astral.sun.sun(locationInfoObj.observer, date=for_date)
    return s["sunset"].astimezone(tz_chicago)

In [ ]:
from screeninfo import get_monitors

In [ ]:
max_height = 0
max_width = 0

for m in get_monitors():
    logging.info(m)
    if m.x + m.width > max_width:
        max_width = m.x + m.width
    if m.y + m.height > max_height:
        max_height = m.y + m.height
logging.info(f"maxes {max_width},{max_height}")

In [ ]:
p = Path("weather.json")
not p.exists() or (
    datetime.now() - datetime.fromtimestamp(p.stat().st_mtime)
).seconds > 60 * 60 * 10

In [ ]:
def isStorming():
    if (
        not p.exists()
        or (datetime.now() - datetime.fromtimestamp(p.stat().st_mtime)).seconds
        > 60 * 60 * 10
    ):
        weather = requests.get(
            "https://dataservice.accuweather.com/forecasts/v1/hourly/1hour/2211489?apikey=lzAwADaLJIEGXGWS4eCsSL7jVGmiBw9l&language=en-us&details=true"
        ).json()
        if "Code" in weather and weather["Code"] == "ServiceUnavailable":
            return False
        p.write_text(json.dumps(weather))
    else:
        weather = json.loads(p.read_text())
    return weather[0]["IconPhrase"] == "Thunderstorms"

In [ ]:
def imgToRatio(h, w):
    if h > w:
        return "16x9"
    elif w > h:
        return "9x16"
    else:
        return "square"

In [ ]:
import pytesseract

offensive_words = requests.get(
    "https://raw.githubusercontent.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/master/en"
).text.splitlines()


def hasOffenseiveText(path):
    text = pytesseract.image_to_string(path).strip()
    for w in offensive_words:
        if w in text:
            return True

    return False

In [ ]:
import cv2
import numpy as np

face_cascade = cv2.CascadeClassifier(
    "/home/srudloff/github/Some-Notebooks/Wallpaper/lbpcascade_animeface.xml"
)


def hasAnimeFace(path):
    print(path)
    # loading in the cascades.

    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    print(f"Found {len(faces)} Anime faces")
    return len(faces) > 0


faceCascade = cv2.CascadeClassifier(
    "/home/srudloff/github/Some-Notebooks/Wallpaper/haarcascade_frontalface_default.xml"
)


def hasHumanFace(path):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30)
        # flags = cv2.CV_HAAR_SCALE_IMAGE
    )

    print(f"Found {len(faces)} human faces!")
    return len(faces) > 0

In [ ]:
def getRandomBackground(size=None, used=None):
    print("grb size", size)
    if used is None:
        used = []
    if size is None:
        size = random.choice(os.listdir(wallpaper_dir))
        logging.info("Selecting random size images")
    try:
        bgs = os.listdir(os.path.join(wallpaper_dir, size))
        if len(bgs) == 0:
            return None
        path = os.path.join(wallpaper_dir, size, random.choice(bgs))
        max_i = 10
        while (
            path in used
            or hasAnimeFace(path)
            or hasHumanFace(path)
            or hasOffenseiveText(path)
        ):
            max_i -= 1
            print("ReRollin", max_i)
            path = os.path.join(wallpaper_dir, size, random.choice(bgs))
            if max_i == 0:
                return None
        return path
    except Exception as e:
        print(e)
        return None

In [ ]:
class ImageSpot:
    def __init__(self, image, x, y, h, w):
        self.image = image
        self.x = x
        self.y = y
        self.height = h
        self.width = w
        print(self)
        self.fillColor = ""
        self.size_data = reduceFraction(h, w)
        self.size_str = f"{self.size_data[0]}x{self.size_data[1]}"
        self.ratio = w / h

    def fill(self, im):
        pos = (self.x, self.y)
        logging.info(f"pasting at {pos}, {im.size}")
        self.image.paste(im, pos)

    def fakeit(self):
        shape = [(self.x, self.y), (self.x + self.width, self.y + self.height)]
        # create rectangle image
        img1 = ImageDraw.Draw(self.image)
        img1.rectangle(shape, fill=self.fillColor)

    def __repr__(self):
        return f"Monitor({self.x},{self.y}) {self.width}X{self.height}"

In [ ]:
im = Image.new(mode="RGBA", size=(max_width, max_height))
image_spots = []
for m in get_monitors():
    image_spots.append(ImageSpot(image=im, x=m.x, y=m.y, h=m.height, w=m.width))


backup_colors = ["#151515", "#131313", "#171717", "#ff00ff"]
needs_more_papers = []
used = []
for item in image_spots:
    logging.info(item)
    wallpaper = None
    wallpaper = getRandomBackground(item.size_str, used)
    logging.info(f"wpf {wallpaper}")
    if wallpaper is None:
        logging.info(f"no image of size:{item.size_str}, faking it")
        needs_more_papers.append(item.size_str)
        item.fillColor = backup_colors.pop(0)
        item.fakeit()
    else:
        img = Image.open(wallpaper)
        logging.info(f"{item} - {item.size_str},{img.size},{wallpaper}")
        item.fill(img)
        used.append(wallpaper)
    print()
wallpaper = os.path.expanduser("~/bg.png")

a = getSunset()
if datetime.now().time().hour > a.hour or isStorming():
    print("Sunset mode")
    enhancer = ImageEnhance.Brightness(im)
    factor = 0.5  # Darkens the image
    im = enhancer.enhance(factor)

im.save(wallpaper)

In [ ]:
get_DBUS_SESSION_BUS_ADDRESS()

In [ ]:
cmd = f"DBUS_SESSION_BUS_ADDRESS='{get_DBUS_SESSION_BUS_ADDRESS()}' /usr/bin/gsettings set org.gnome.desktop.background picture-uri {wallpaper}"
logging.debug(cmd)
os.system(cmd)

cmd = f"DBUS_SESSION_BUS_ADDRESS='{get_DBUS_SESSION_BUS_ADDRESS()}' /usr/bin/gsettings set org.gnome.desktop.background picture-uri-dark {wallpaper}"
logging.debug(cmd)
os.system(cmd)

try:
    os.unlink("wp-lock")
except Exception:
    pass

cmd = f"DBUS_SESSION_BUS_ADDRESS='{get_DBUS_SESSION_BUS_ADDRESS()}' /usr/bin/gsettings set org.gnome.desktop.background picture-options spanned"
os.system(cmd)